In [2]:
from matplotlib import pyplot as plt
from IPython import display
import numpy as np
import random
from matplotlib import colors
import copy
import time
import scipy.optimize

%matplotlib inline

In [3]:
# assumes periodic BCs
def metropolis_potts(init, a, boltz):
    """Returns a grid evolved one step in the Potts model using the Metropolis algorithm"""
    
    grid = copy.copy(init)
    
    X = np.shape(grid)[1]
    Y = np.shape(grid)[0]
    
    new_state = random.randint(0,q)
    
    # select random spin from the input spin grid
    x = random.choice(range(X))
    y = random.choice(range(Y))
    
    # checking the energy cost
    energy_i = 0
    energy_f = 0
    
    if grid[y, (x+1) % X] == grid[y,x]:
        energy_i += 1
        
    if grid[y, (x-1) % X] == grid[y,x]:
        energy_i += 1
        
    if grid[(y + 1) % Y, x] == grid[y,x]:
        energy_i += 1
        
    if grid[(y-1) % Y, x] == grid[y,x]:
        energy_i += 1
    
    if grid[y, (x+1) % X] == new_state:
        energy_f += 1
        
    if grid[y, (x-1) % X] == new_state:
        energy_f += 1
        
    if grid[(y + 1) % Y, x] == new_state:
        energy_f += 1
        
    if grid[(y-1) % Y, x] == new_state:
        energy_f += 1
        
    d_energy = int(-1*(energy_f - energy_i))
    
    
    # applying the metropolis algorithm
    if d_energy <= 0:
        grid[y,x] = new_state
        
    elif random.uniform(0,1) < boltz[str(abs(d_energy))]:
        grid[y,x] = new_state
        
    else:
        d_energy = 0
        

    return [grid, d_energy]

In [183]:
# assumes periodic boundary condition
# it's possible to compute the boltzmann factors in-function, but that makes it worse for looping
def heatbath_potts(init, q, boltz):
    """Returns a grid evolved one step in the Potts model using the heat-bath algorithm at temperature T"""
    
    grid = copy.copy(init)
    
    X = np.shape(grid)[1]
    Y = np.shape(grid)[0]
    
    # select random spin from the input spin grid
    x = random.choice(range(X))
    y = random.choice(range(Y))
    
    print((y, x))
    
    # compute the relevant energies for the heat-bath algorithm (only nearest neighbor interactions)
    # we only want to compute the energies that are non-zero
    check = []
    check.append(grid[(y+1) % Y, x])
    check.append(grid[(y-1) % Y, x])
    check.append(grid[y % Y, (x+1) % X])
    check.append(grid[y % Y, (x-1) % X])
    check = list(set(check))
    
    
    print(check)
    
    # initialize the probabilities to 1, because the zero energies have boltzmann factor equal to 1
    probs = np.ones(q)
    
    # replace the appropriate elements with the boltzman factors; note that order in probs is important!
    for c in check:
        e = potts_energy_spin(grid, [y,x], c)
        probs[c-1] = boltz[str(e)]
        
    probs = np.array(probs)/np.sum(probs)   
    
    print(probs)
    
    new_state = np.random.choice(range(1, q+1), p = probs)
    
    grid[y,x] = new_state
    
    return grid

    

In [184]:
T = 1
boltz = {'-1' : np.exp(1/T), '-2' : np.exp(2/T), '-3' : np.exp(3/T), '-4' : np.exp(4/T)}

a = np.array([[1, 2, 1], [1, 2, 1], [1, 2, 3]])

heatbath_potts(a, 3, boltz)

(1, 2)
[1, 2, 3]
[0.57611688 0.21194156 0.21194156]


array([[1, 2, 1],
       [1, 2, 2],
       [1, 2, 3]])

In [46]:
def potts_energy_spin(init, spin, q = None):
    """Compute the energy of a spin on a Potts lattice"""
    
    grid = copy.copy(init)
    
    X = np.shape(grid)[1]
    Y = np.shape(grid)[0]
    
    x = spin[1]
    y = spin[0]
    
    # allows you to change the value of the spin in question
    if q is not None:
        grid[y,x] = q
    
    energy = 0
    
    if grid[y, (x + 1) % X] == grid[y,x]:
        energy += -1

    if grid[y, (x - 1) % X] == grid[y,x]:
        energy += -1

    if grid[(y + 1) % Y, x] == grid[y,x]:
        energy += -1

    if grid[(y - 1) % Y, x] == grid[y,x]:
        energy += -1
    
    return energy
    
    
    

In [38]:
def potts_energy(init):
    """Computes the energy of a Potts grid"""
    
    grid = copy.copy(init)
    
    X = np.shape(grid)[1]
    Y = np.shape(grid)[0]
    
    energy = 0
    
    for i in range(Y):
        for j in range(X):
            spin_energy = 0
        
            if grid[i, (j + 1) % X] == grid[i,j]:
                spin_energy += -1

            if grid[i, (j - 1) % X] == grid[i,j]:
                spin_energy += -1

            if grid[(i + 1) % Y, j] == grid[i,j]:
                spin_energy += -1

            if grid[(i - 1) % Y, j] == grid[i,j]:
                spin_energy += -1
                
            energy += spin_energy
            
    return energy
    
    

In [32]:
def plot_potts_grid(init, title):
    """Creates a discrete colormap for an input Potts grid, red for up (1) and blue for down (-1)"""
    X = np.shape(init)[1]
    Y = np.shape(init)[0]
    
    cmap = colors.ListedColormap(['blue', 'red'])
    bounds = [0,1,1]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    
    fig, ax = plt.subplots(figsize=(8,8))
    plt.imshow(init, cmap=cmap, norm=norm)
    plt.title(title, fontsize=20)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

    display.clear_output(wait = True)

    plt.show()